In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style = "font-size : 50px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : #bedcfa; border-radius: 5px 5px;"><strong>Heart Attack EDA and Prediction</strong></p>

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Importing Libraries</strong></p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Read The Dataset</strong></p>

In [1]:
heart=pd.read_csv("../input/heart-attack-analysis-prediction-dataset/heart.csv")
saturation=pd.read_csv("../input/heart-attack-analysis-prediction-dataset/o2Saturation.csv")

In [1]:
heart.head()

In [1]:
saturation.head()

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Data Description</strong></p>

* **age - Age of the patient**
* **sex - Sex of the patient** 
* **cp - Chest pain type ~ 0 = Typical Angina, 1 = Atypical Angina, 2 = Non-anginal Pain, 3 = Asymptomatic**
* **trtbps - Resting blood pressure (in mm Hg)** 
* **chol - Cholestoral in mg/dl fetched via BMI sensor**
* **fbs - (fasting blood sugar > 120 mg/dl) ~ 1 = True, 0 = False** 
* **restecg - Resting electrocardiographic results ~ 0 = Normal, 1 = ST-T wave normality, 2 = Left ventricular hypertrophy** 
* **thalachh - Maximum heart rate achieved**
* **oldpeak - Previous peak**
* **slp - Slope** 
* **caa - Number of major vessels**
* **thall - Thalium Stress Test result ~ (0,3)** 
* **exng - Exercise induced angina ~ 1 = Yes, 0 = No** 
* **output - target : 0= less chance of heart attack 1= more chance of heart attack**

In [1]:
heart.shape

In [1]:
col=heart.columns
col

In [1]:
heart.describe()

In [1]:
heart.nunique()

In [1]:
heart

In [1]:
categorical_cols = ['sex','exng','caa','cp','fbs','restecg','slp','thall']
numerical_cols = ["age","trtbps","chol","thalachh","oldpeak"]
target_col = ["output"]

In [1]:
# check for the missing values
heart.isnull().sum()

No missing values are found

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Univariate Analysis</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Categorical and Target features</strong></p>

In [1]:
# target variable
heart['output'].value_counts(normalize=True).plot.bar(color=['red','blue'],edgecolor='black',title='target variable')

* **Around 55% people have more chances to get heart attack**
* **Around 45% people have less chances to get heart attack**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Sex Feature</strong></p>

In [1]:
# sex variable
heart['sex'].value_counts(normalize=True).plot.bar(color=['cyan','magenta'],edgecolor='black',title='sex variable')

* **Around 68 % people are with sex=1**
* **Around 30 % people are with sex=0**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Chest Pain Feature</strong></p>

In [1]:
# cp variable
heart['cp'].value_counts(normalize=True).plot.bar(color=['yellow','orange','cyan','magenta'],edgecolor='black',title='chest pain variable')

* **Around 50 % of the people have chest pain type: Typical Angina**
* **Around 28 % of the people have chest pain type: Non-anginal Pain**
* **Around less than 20 % of the people have chest pain type: Atypical Angina**
* **Around less than 10% of the people have chest pain type: Asymptomatic**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>1.exercise induced angina</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>2.fasting blood sugar > 120 mg/dl</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>3.resting electrocardiographic results</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>4. Slope</strong></p>

In [1]:
plt.figure(figsize=(20,7))
plt.subplot(221)
heart['exng'].value_counts(normalize=True).plot.bar(color=['yellow','orange'],edgecolor='black',title='exercise induced angina (1 = yes; 0 = no)')
plt.subplot(222)
heart['fbs'].value_counts(normalize=True).plot.bar(color=['yellow','green'],edgecolor='black',title='(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)')
plt.subplot(223)
heart['restecg'].value_counts(normalize=True).plot.bar(color=['magenta','blue','cyan'],edgecolor='black',title='resting electrocardiographic results')
plt.subplot(224)
heart['slp'].value_counts(normalize=True).plot.bar(color=['red','blue','green'],edgecolor='black',title='- Slope')

* **More than 65 % of the people Exercise don't induced angina**
* **More than 35 % of the people Exercise induced angina**
* **less than 20 % of the people have fasting blood sugar > 120 mg/dl**
* **More than 80 % of the people have fasting blood sugar <= 120 mg/dl**
* **less than 50 % of the people have resting electrocardiographic results normal**
* **50 % of the people have resting electrocardiographic results: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)**
* **1% or 2% of the people have resting electrocardiographic results: showing probable or definite left ventricular hypertrophy by Estes' criteria**


<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>1.number of major vessels</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>2.Thalium Stress Test result ~ (0,3)</strong></p>

In [1]:
plt.figure(figsize=(20,7))
plt.subplot(221)
heart['caa'].value_counts(normalize=True).plot.bar(color=['magenta','blue','cyan','red','orange'],edgecolor='black',title='number of major vessels')
plt.subplot(222)
heart['thall'].value_counts(normalize=True).plot.bar(color=['lightblue','lightgreen','lightyellow','magenta'],edgecolor='black',title='Thalium Stress Test result ~ (0,3)')

<p style = "font-size : 30px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Numerical features</strong></p>

In [1]:
numerical_cols

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>age , blood pressure , cholestoral , Heart Rate</strong></p>

In [1]:
plt.figure(figsize=(20,7))
plt.subplot(221)
heart['age'].plot.hist(edgecolor='black',color='lightgreen',title='age variable')
plt.subplot(222)
heart['trtbps'].plot.hist(edgecolor='black',color='lightblue',title='resting blood pressure in mm hg')
plt.subplot(223)
heart['chol'].plot.hist(edgecolor='black',color='lightcoral',title='cholestoral in mg/dl fetched via BMI sensor')
plt.subplot(224)
heart['thalachh'].plot.hist(edgecolor='black',color='lightgray',title='maximum heart rate achieved')

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Oldpeak</strong></p>

In [1]:
heart['oldpeak'].plot.hist(edgecolor='black',color='lightyellow',title='oldpeak variable')

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Bivariate Analysis</strong></p>

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> effect of age on heart attack</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of age on heart attack")
sns.lineplot(x=heart['age'],y=heart['output'])


* > **The people with the age 30 to 35 have higher chance of heart attacks**
* > **The people with the age than 70 and less than 75 have higher chance of heart attacks** 
* > **apart from it no certain trend i will be able to find**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> heart attack related with sex</strong></p>

In [1]:
sns.countplot(data=heart,x='sex',palette=["blue","red"], hue='output')

**people of sex=1 have higher chances of getting heart attacks**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> heart attack related with chest pain</strong></p>

In [1]:
sns.kdeplot(data=heart, x='cp',hue="output", fill=True)

**people with chest pain type=2 have higher chance of getting heart attacks**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> heart attack related with age</strong></p>

In [1]:
sns.kdeplot(data=heart, x='age',hue="output", fill=True)

**according to the data people with lower age have more chances of getting heart attacks than those of higher ages**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> heat attack realted with thalium stress test</strong></p>

In [1]:
sns.kdeplot(data=heart, x='thall',hue="output", fill=True)

**people with thall test=2 have higher chance of getting heart attacks**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> heat attack realted with Exercise induced angina</strong></p>

In [1]:
sns.kdeplot(data=heart, x='exng',hue="output", fill=True)

**people with exng=0 have higher chances of getting heart attacks**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> effect of age on blood pressure</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of age on blood pressure")
sns.lineplot(x=heart['age'],y=heart['trtbps'])


* > **as age is incresing the increase in the blood pressure has been founded**


<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> effect of age on cholestrol level</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of age on cholestrol level")
sns.lineplot(x=heart['age'],y=heart['chol'])

* > **as age is incresing the increase in the cholestrol level has been founded**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> effect of age on heart rate</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of age on heart rate")
sns.lineplot(x=heart['age'],y=heart['thalachh'])

* > **as age is incresing the decrease in the heart rate has been founded**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> How does incresed heart rate and age affect the heart attack</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of heart attack with increase in age and heart rate")
sns.lineplot(x=heart['age'],y=heart['thalachh'],hue=heart['output'])

* **as with the increase in the age the heart rate is decresing and also the people with more chances of heart attacks are decreasing hence we can say higher heart rate increases the chance of heart attack**

In [1]:
sns.kdeplot( data=heart, x='thalachh',hue="output",fill=True)

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> How does incresed cholestrol and age affect the heart attack</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of heart attack with increase in age and cholestrol")
sns.lineplot(x=heart['age'],y=heart['chol'],hue=heart['output'])

* **as with the increase in the age the cholestrol level is incresing and also the people with more chances of heart attacks are also increasing hence we can say higher cholestrol level increases the chance of heart attack**

In [1]:
sns.kdeplot( data=heart, x='chol',hue="output",fill=True)

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong> How does incresed blood pressure and age affect the heart attack</strong></p>

In [1]:
plt.figure(figsize=(10,7))
plt.style.use("fivethirtyeight")
plt.title("effect of heart attack with increase in age and blood pressure")
sns.lineplot(x=heart['age'],y=heart['trtbps'],hue=heart['output'])

* **as with the increase in the age the blood pressure is incresing and also the people with more chances of heart attacks are also increasing hence we can say blood pressure increases the chance of heart attack**

In [1]:
sns.kdeplot( data=heart, x='trtbps',hue="output",fill=True)

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Model Building</strong></p>

In [1]:
target=heart['output']
target

In [1]:
heart.drop(['output'],axis=1,inplace=True)
heart.head()

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Checking for skewness</strong></p>

In [1]:
heart['trtbps'].plot(kind='density')
plt.show()
heart['chol'].plot(kind='density')
plt.show()
heart['thalachh'].plot(kind='density')
plt.show()
heart['age'].plot(kind='density')
plt.show()

In [1]:
heart.head(1)

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Robust Scaler</strong></p>

In [1]:
from sklearn import preprocessing
scaler = preprocessing.RobustScaler()
robust_df = scaler.fit_transform(heart)
robust_df = pd.DataFrame(robust_df, columns =['age','sex','cp','trtbps','chol','fbs','restecg','thalachh','exng','oldpeak','slp','caa','thall'])
robust_df

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Standard Scaler</strong></p>

In [1]:
scaler = preprocessing.StandardScaler()
standard_df = scaler.fit_transform(robust_df)
standard_df = pd.DataFrame(standard_df, columns =['age','sex','cp','trtbps','chol','fbs','restecg','thalachh','exng','oldpeak','slp','caa','thall'])

In [1]:
standard_df.head()

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : orange; border-radius: 5px 5px;"><strong>Train Test Split</strong></p>

In [1]:
x_train,x_test,y_train,y_test=train_test_split(heart,target,test_size=0.1,random_state=42)

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Logistic Regresson</strong></p>

In [1]:
logistic=LogisticRegression(max_iter=100,random_state=1,n_jobs=-1)
logistic.fit(x_train,y_train)
pred1=logistic.predict(x_test)
pred1

In [1]:
logistic.score(x_train,y_train)*100

In [1]:
logistic.score(x_test,y_test)*100

In [1]:
from sklearn.metrics import accuracy_score

print('Logistic Regresson model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred1)))

In [1]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
d_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train,y_train)*100,2)
print(f'{acc_decision_tree}%')

In [1]:
from sklearn.metrics import accuracy_score

print('Decision Tree model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, d_pred)))

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>LightGBM</strong></p># Lightgbm

In [1]:
import lightgbm as lgb
lgbm= lgb.LGBMClassifier()
lgbm.fit(x_train,y_train)
pred2=lgbm.predict(x_test)
acc_lgbm=round(lgbm.score(x_train,y_train)*100,2)
print(f'{acc_lgbm}%')

In [1]:
from sklearn.metrics import accuracy_score

print('LightGBM model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred2)))

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>XGBoost</strong></p>

In [1]:
import xgboost as xgb
# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=heart,label=target)

**DMatrix is an internal data structure that is used by XGBoost, which is optimized for both memory efficiency and training speed.**

In [1]:
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 0.01,
            'n_estimators':100
        }         

In [1]:
import xgboost as xgb
xgbo= xgb.XGBClassifier(**params)
xgbo.fit(x_train,y_train)
pred3=xgbo.predict(x_test)
acc_xgbo=round(xgbo.score(x_train,y_train)*100,2)
print(f'{acc_xgbo}%')

In [1]:
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, pred3)))

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>XGBoost with Cross Validation</strong></p>

In [1]:
# cross validation
from xgboost import cv

params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

In [1]:
xgb_cv.head()

In [1]:
xgb_cv.shape

In [1]:
accuracy_xgb=xgb_cv["test-auc-mean"][49]
print(accuracy_xgb)

**The accuracy is increased from 0.8387 to 0.9028**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Insights how model has arrived at its final decision</strong></p>

In [1]:
xg_reg = xgb.train(params=params,dtrain=data_dmatrix, num_boost_round=10)

In [1]:
xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 20]
plt.show()

**These plots provide insight into how the model arrived at its final decisions and what splits it made to arrive at those decisions.**

<p style = "font-size : 25px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightgreen; border-radius: 5px 5px;"><strong>Feature Importance</strong></p>

In [1]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [6, 6]
plt.show()

>**The most imporatant feature of the dataset is age and the less important feature id chest pain and thalachh**

<p style = "font-size : 35px; color : #532e1c ; font-family : 'Comic Sans MS'; text-align : center; background-color : lightblue; border-radius: 5px 5px;"><strong>Score Comparison</strong></p>

In [1]:
models = pd.DataFrame({
    'Model' : ['Logistic Regression','Decision Tree', 'Lightgbm', 'XgBoost','XgBoost with cross validation'],
    'Score' : [accuracy_score(y_test, pred1)*100,accuracy_score(y_test, d_pred)*100,accuracy_score(y_test, pred2)*100,accuracy_score(y_test, pred3)*100,accuracy_xgb*100]
})


models.sort_values(by = 'Score', ascending = False)